# Iris Dataset (builtin dataset)
•The dataset contains a set of 150 records under five attributes sepal
length, sepal width, petal length, petal width and species.

•This data sets consists of 3 different types of irises’ ( Setosa , Versicolour , and Virginica) petal and sepal length, stored in a 150x4 numpy.ndarray.

•The rows being the samples and the columns being: Sepal Length, Sepal Width,
Petal Length and Petal Width.

In [ ]:
import sklearn as sk
from sklearn.datasets  import load_iris
import numpy as np
import pandas as pd
import matplotlib as plt
import random

In [ ]:
#function to convert sklearn dataset to dataFrame

def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [ ]:
#load data from sklearn
iris = sklearn_to_df(sk.datasets.load_iris())
iris.head()

iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
#split the labels and data
X = iris.iloc[:, :-1].values
y = iris.iloc[:, -1].values

In [ ]:
# function to split train & test & validate

def train_validate_test_split (data , labels,testRatio,valRatio):
  #convert the data to numpy array
  data = np.array(data)
  labels = np.array(labels)
  #shuffle data
  indices = np.arange(data.shape[0])
  np.random.shuffle(indices)

  data = data[indices]
  labels = labels[indices]

  #size of data
  size =  labels.shape[0]

  #split train data
  train_size = (1-testRatio-valRatio)*size
  train_size = int(train_size)
  train_X = data[:train_size]
  train_y = labels[:train_size] 

  #split test data
  test_size = int(testRatio*size)
  test_X = data[train_size+1 : train_size+test_size+1]
  test_y = labels[train_size+1 : train_size+test_size+1]

  #split validate data
  val_size = int(valRatio*size)
  val_X = data[train_size+test_size+1:]
  val_y = labels[ train_size+test_size+1:]

  return train_X, test_X, val_X, train_y, test_y, val_y




In [ ]:
train_X, test_X, val_X, train_y, test_y, val_y = train_validate_test_split(X,y,0.3,0.3)
print(train_X.shape[0])
print("--------------")
print(train_y.shape[0])
print("--------------")
print(test_X.shape[0])
print("--------------")
print(test_y.shape[0])
print("--------------")
print(val_X.shape[0])
print("--------------")
print(val_y.shape[0])
print("--------------")



59
--------------
59
--------------
45
--------------
45
--------------
45
--------------
45
--------------


In [ ]:
#function to calculate accuracy of Prediction results
def calculate_accuracy (predicted_y ,y):
  #convert data to numpy array
  y = np.array(y)
  predicted_y = np.array(predicted_y)
  #size of data
  size = y.shape[0]
  #count the correct Prediction results
  cnt =0
  for i in range(0,size,1):
    if y[i]==predicted_y[i]:
      cnt+=1
  #calculate accuracy
  return cnt/size

In [ ]:
acc = calculate_accuracy(test_y,val_y)
print(acc)

0.37777777777777777
